# Examples of utils provided by ReGraph's Neo4jHierarchy module

In [1]:
import networkx as nx
from regraph import Rule, plot_rule

from regraph.neo4j.hierarchy import Neo4jHierarchy
import regraph.neo4j.cypher_utils as cypher
import regraph.neo4j.result_utils as rs

## Initializing Neo4j database

0. When installing neo4j you will be asked to choose login/password for you dbs (here its "neo4j"/"admin"), if you choose other -- change in the cell below.
1. To start neo4j server run `sudo service neo4j start`
2. Check status by running `sudo service neo4j status`. Here you can check the _bolt_ port, change the cell below if different from 7687
3. You can query the db by using the neo4j browser, the address can be found also in the result of 'status', e.g. "Remote interface available at http://localhost:7474/".

In [2]:
# initialize the neo4j driver, wrappped into Neo4jHierarchy object
h = Neo4jHierarchy(uri="bolt://localhost:7687", user="neo4j", password="admin")

In [3]:
#here we clear the hierarchy
h.clear()
#

## Hierarchy

A hierarchy is represented by at least 2 graphs with a typing relation (homorphism) between them.


Each graph in the database is represented by a unique label which enables us to match quickly the nodes of this graph. Each node of a graph has then the labels *node* and *graphLabel*. We can match these nodes by using the query:

> MATCH (n:node:graphLabel) RETURN n


The edges of a graph are labeled as *edge* and the typing edges are labeled as *typing*. We can then easily find the image of a node with the query:

> OPTIONAL MATCH (n:node:graphLabel)-[:typing]->(m) RETURN m

The hierarchy skeleton is represented by nodes labeled as hierarchyNodes. It enables us to know which graphs are the ancestors of a graph we want to rewrite for example.

## Adding a graph to the hierarchy (Action Graph)

Here we create a first graph called *ActionGraph*. This graph represent people and the relationship between them.

In [4]:
h.add_graph('actionGraph')

In [5]:
ag = h.access_graph('actionGraph')

In [6]:
nodes = [
    ("a", {"name": "Jack", "age": 43, "hobby": {"hiking", "music"}}), 
    ("b", {"name": "Robert", "age": 35, "hobby": {"sport", "music"}, "height": 178}),
    ("c", {"name": "Paul", "age": 18}),
    ("d", {"name": "Joe"}),
    ("e", {"name": "Paul"}),
    ("f", {"name": "Bob"})
]
edges = [
    ("a", "b", {"type": {"friends"}}), 
    ("a", "c", {"type": {"colleagues"}}),
    ("b", "c", {"type": "colleagues"}), 
    ("d", "a", {"type": "child_of"}), 
    ("d", "e", {"type": "enemies"}),
    ("e", "f", {"type": "friends"}), 
    ("f", "d", {"type": "friends"}),
    ("e", "e")
]
ag.add_nodes_from(nodes)
ag.add_edges_from(edges)

## Adding a second graph to the herarchy (Meta Model)

Here we create a second graph called *MetaModel*. Which represent the types of people (*Adult* or *Child*) and the relationships between them.

In [7]:
h.add_graph('metaModel')

In [8]:
mm = h.access_graph('metaModel')

In [9]:
nodes = [
    ("a", {"type": "Adult"}),
    ("b", {"type": "Child"})
]
edges = [
    ("a", "a", {"type": {"friends", "colleagues"}}),
    ("b", "a", {"type": {"child_of"}}),
    ("b", "b", {"type": {"friends", "enemies"}})
]
mm.add_nodes_from(nodes)
mm.add_edges_from(edges)

## Typing

Here we create a typing of the *ActionGraph* on the *MetaModel*.

In [10]:
mapping = {
    "a":"a",
    "b":"a",
    "c":"a",
    "d":"b",
    "e":"b",
    "f":"b"
}
h.add_typing('actionGraph', 'metaModel', mapping)

In [11]:
h.check_typing('actionGraph', 'metaModel')

True


## Classic update functions

We still can use the modification functions on these grapphs.

In [ ]:
#ag.merge_nodes1(["b", "c"])

In [ ]:
#ag.clone_node("e")

## Graph rewriting and propagation up

In [ ]:
pattern = nx.DiGraph()
pattern.add_nodes_from(["x", "y"])
pattern.add_edges_from([("y", "x"), ("y", "y")])

In [ ]:
rule = Rule.from_transform(pattern)
clone_name1, _ = rule.inject_clone_node("y")
rule.inject_remove_edge(clone_name1, "y")
rule.inject_remove_node("x")

In [ ]:
rule.removed_edges()

In [ ]:
plot_rule(rule)

In [ ]:
instances = mm.find_matching(pattern)
print("Instances: ", instances)

In [ ]:
#h.rewrite('metaModel', rule, instances[0])
h.rewrite_v2('metaModel', rule, instances[0])

In [ ]:
h.propagation_up('metaModel')

## Graph rewritting and propagation down

In [ ]:
pattern = nx.DiGraph()
pattern.add_nodes_from(["w", "x", "y", "z"])
pattern.add_edges_from([("x", "w"), ("y", "x"), ("y", "z"), ("z","z")])

In [ ]:
rule = Rule.from_transform(pattern)
rule.inject_add_node("new_node")
rule.inject_add_edge("new_node", "y")
rule.inject_merge_nodes(["w", "z"])

In [ ]:
plot_rule(rule)

In [ ]:
instances = ag.find_matching(pattern)
print("Instances: ", instances)

In [ ]:
#h.rewrite('actionGraph', rule, instances[1])
h.rewrite_v2('actionGraph', rule, instances[1])

In [ ]:
h.propagation_down('actionGraph')

## Removing a graph from the hierarchy

We can also remove a graph from the hierarchy. All its nodes and connections (edges and typing edges) are removed.

In [ ]:
#h.remove_graph('MetaModel')